### Description
We model the prediction using a transformer decoder, we train GPT-2 to predict the next token. The trick is, that we let every sentence be of the form:
\begin{align}
    &["1","5/2","/","0","-1", "[SEPARATOR]", "<MASK>"] \\ 
    &--- \textrm{Predicted that} "<MASK>" = "Z" ---> \\
    &["1","5/2","/","0","-1", "[SEPARATOR]", "Z", "<MASK>"]
\end{align}
where "[SEPARATOR]" indicates that we're not dealing with the polynomial anymore, we're dealing with the 'answer'. This process is continued iteratively untill the max-length is reached or the network predicts an EOS token

In [109]:
# External imports
import torch

# For saving and publishing models and datasets to the huggingface hub
from huggingface_hub import notebook_login, create_repo

from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, DownloadConfig, IterableDataset

from tqdm.notebook import tqdm
import linecache
import time
import json

In [110]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\Hugo\.cache\huggingface\token
Login successful


In [112]:
# Domestic imports
from model.tokenize_input import input_string_to_tokenize_expression
from model.tokens import TOKEN_TYPE_ANSWERS, TOKEN_TYPE_EXPRESSIONS
from model.equation_interpreter import Equation
from model.vocabulary import Vocabulary

In [114]:
# Create a combined vocabulary
vocabulary = Vocabulary.construct_from_list(TOKEN_TYPE_EXPRESSIONS + TOKEN_TYPE_ANSWERS)
vectorized_sample = vocabulary.vectorize(["#", "/", "0", "-1", "[SEP]", "TT_INTEGER"])
vectorized_sample, [vocabulary.getToken(idx) for idx in vectorized_sample]

([64, 40, 39, 19, 9, 67, 49, 65],
 ['<BEGIN>', '#', '/', '0', '-1', '[SEP]', 'TT_INTEGER', '<END>'])

In [115]:
model_name = "JustSumAI"
repo_name = f"{model_name}_cleaned_gpt2_data"

# Dataset creation
Takes `input_file_answers` and `input_file_expressions` and combines every convertable version (i.e. does not contain PolyGamma or similar) of these, separated by a `"[SEP]"` token

In [116]:
# Number of evaluation datapoints
num_eval_points = 20

In [117]:
# Paths to data files
input_file_answers = "./data/answers-1000.txt"
input_file_expressions = "./data/expressions-1000.txt"

In [131]:
max_length = 0

# Make file for cleaned data
cleaned_gpt2_data_train = "/".join(input_file_answers.split("/")[:-1]) + "/cleaned_gpt2_data_train.txt"
cleaned_gpt2_data_eval = "/".join(input_file_answers.split("/")[:-1]) + "/cleaned_gpt2_data_eval.txt"

# Run through every equation in the input files and delete rows with unknown tokens
dataset_size = sum(1 for _ in open(input_file_answers, 'rb'))

# Open and populate cleaned files and
f_cleaned_train = open(cleaned_gpt2_data_train, "w")
f_cleaned_eval = open(cleaned_gpt2_data_eval, "w")

start_time = time.time()
n_cleaned = 0
flag = False
for line_number in tqdm(range(1,dataset_size+1)):
    # Flag determines whether the data point is an evaluation or a training data point
    flag = flag or (line_number-1) % (dataset_size//num_eval_points) == 0
    
    # Get corresponding equation and expression
    raw_equation = linecache.getline(input_file_answers, line_number)
    raw_expression = linecache.getline(input_file_expressions, line_number)

    # Skip line if aborted error
    if raw_equation == "$Aborted\n": continue

    # Construct equation and convert to postfix
    equation = Equation.makeEquationFromString(raw_equation)
    if not equation.tokenized_equation: continue
    equation.convertToPostfix()
    if equation.notation == "infix": continue

    # Vectorize corresponding answer (without SOS) and expression (without EOS)
    vectorized_answers = vocabulary.vectorize([token.t_type for token in equation.tokenized_equation])[1:]
    vectorized_expressions = vocabulary.vectorize([str(token) for token in input_string_to_tokenize_expression(raw_expression)])[:-1]

    max_length = max(max_length, len(vectorized_expressions) + len(vectorized_answers) + 1)

    if flag:
        flag = False
        f_cleaned_eval.write(f"{json.dumps(vectorized_expressions + [vocabulary.getIndex('[SEP]')] + vectorized_answers)}\n")
    else:
        f_cleaned_train.write(f"{json.dumps(vectorized_expressions + [vocabulary.getIndex('[SEP]')] + vectorized_answers)}\n")

    # Write them to cleaned data file and separate them by [SEP] token
    n_cleaned += 1
    
f_cleaned_train.close()
f_cleaned_eval.close()

print(f"\nStats: \n--- Initial size: {dataset_size} \n--- Time since start: {round(time.time() - start_time, 4)} s \n--- Successes: {n_cleaned} \n--- Fails: {line_number - n_cleaned + 1} \n--- Max length: {max_length}")

  0%|          | 0/1000 [00:00<?, ?it/s]

Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [PolyGamma]
Unknown token encountered: [RootSum]
Unknown token encountered: [PolyGamma]
Unknown token encountered: 

In [132]:
with open(f"./data/metadata.txt", "w") as f:
    f.write(json.dumps({"max_length": max_length}))
f.close()

As per: https://huggingface.co/docs/datasets/create_dataset

Make a dataset from an iterator and the push it to the hub

In [133]:
processed_ds_size = sum(1 for _ in open(cleaned_gpt2_data_train, 'rb')) + sum(1 for _ in open(cleaned_gpt2_data_eval, 'rb'))

# Sanity check
print(processed_ds_size,sum(1 for _ in open(cleaned_gpt2_data_train, 'rb')), sum(1 for _ in open(cleaned_gpt2_data_eval, 'rb')))
assert processed_ds_size == n_cleaned

processed_ds_size

336 316 20


336

In [136]:
dataset = load_dataset("/".join(cleaned_gpt2_data_train.split("/")[:-1]), 
             data_files={"train": cleaned_gpt2_data_train.split("/")[-1], "validation": cleaned_gpt2_data_eval.split("/")[-1]})
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to C:/Users/Hugo/.cache/huggingface/datasets/text/data-60c7b7980a829936/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 316
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 20
    })
})

In [137]:
create_repo(repo_name, exist_ok=True, repo_type="dataset")

RepoUrl('https://huggingface.co/datasets/Dragonoverlord3000/JustSumAI_cleaned_gpt2_data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Dragonoverlord3000/JustSumAI_cleaned_gpt2_data')

In [138]:
dataset.push_to_hub(repo_name)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split validation to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [81]:
dataset = load_dataset(f"Dragonoverlord3000/{repo_name}")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/336 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/Hugo/.cache/huggingface/datasets/Dragonoverlord3000___parquet/Dragonoverlord3000--JustSumAI_cleaned_gpt2_data-7bb197e8e347c6a0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [82]:
json.loads(dataset["train"][0]["text"])

[64,
 9,
 39,
 25,
 22,
 21,
 0,
 4,
 26,
 23,
 67,
 51,
 49,
 49,
 49,
 61,
 48,
 59,
 49,
 61,
 51,
 49,
 49,
 61,
 59,
 48,
 58,
 49,
 61,
 58,
 49,
 48,
 60,
 49,
 61,
 58,
 49,
 44,
 60,
 49,
 61,
 58,
 49,
 49,
 53,
 60,
 44,
 60,
 49,
 61,
 59,
 49,
 49,
 49,
 49,
 61,
 49,
 53,
 49,
 61,
 58,
 61,
 53,
 60,
 44,
 60,
 49,
 61,
 58,
 49,
 44,
 60,
 49,
 49,
 49,
 61,
 49,
 53,
 49,
 61,
 58,
 53,
 60,
 61,
 59,
 49,
 49,
 57,
 60,
 49,
 61,
 59,
 49,
 49,
 57,
 60,
 49,
 61,
 58,
 49,
 49,
 57,
 60,
 49,
 61,
 59,
 49,
 49,
 49,
 53,
 59,
 49,
 61,
 57,
 60,
 49,
 61,
 59,
 49,
 49,
 53,
 60,
 49,
 49,
 53,
 59,
 49,
 61,
 57,
 60,
 49,
 61,
 59,
 49,
 49,
 49,
 53,
 58,
 49,
 61,
 57,
 60,
 49,
 61,
 59,
 49,
 49,
 53,
 60,
 49,
 49,
 53,
 58,
 49,
 61,
 57,
 60,
 49,
 61,
 58,
 60,
 59,
 65]

# Training GPT-2 from scratch

### Loading the model
The model is GPT-2 with 1.5 BILLION  parameters!!!

In [99]:
config = AutoConfig.from_pretrained("gpt2-xl", vocab_size=len(vocabulary))
model = AutoModelForCausalLM.from_config(config)
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(68, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1600, out_features=68, bias=False)
)

In [100]:
f"GPT-2 (xl) size: {model.num_parameters()/1_000_000:.1f}M parameters"

'GPT-2 (xl) size: 1477.3M parameters'

In [103]:
# Save newly initialized model - takes a while
model.save_pretrained(f"models/{model_name}", 
                      push_to_hub=True, 
                      organization="Dragonoverlord3000")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/5.96G [00:00<?, ?B/s]